# Chiller Model

In this tutorial we want to create a prosumer model for asessing the heat production (delivered power), and the power consumption of a ater-coooler compressor chiller.

## Chiller Calculation

Our model uses CoolProp data to calculate a simple refrigeration cycle, and it makes no assumptions about compressor speed, so it will work better with an all or nothing chiller (although it can be tuned to calculate a variable speed chiller approximately).

First, some imports:

In [1]:
import os,sys
sys.path.insert(1, r"/Users/vincenzonavavnava/Documents/SENERGY/pandaprosumer") #to be removed later

import pandas as pd
import matplotlib.pyplot as plt
import pyromat as pm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


from pandaprosumer.prosumer.senergy_nets.chiller import chiller
from pandapower.timeseries.data_sources.frame_data import DFData

from pandaprosumer.create import (
    create_empty_prosumer_container,
    define_location,
    define_period,
    define_senergy_nets_chiller,
)

from pandaprosumer.run_time_series import run_timeseries

from pandaprosumer.controller.senergy_nets import (
    ChillerControllerData,
    ChillerController,
)


integrate microscope first!
integrate microscope first!
integrate microscope first!
integrate microscope first!
integrate microscope first!


In this example, we create a heat demand for a temperature time-series for 1st Jan 2020, with a 60min resolution, using a support spreadsheet for the input time series:

* Set Point Temperature T_set [K]	

* Evaporator inlet temperature T_in_ev [K]

* Condenser inlet temperature T_in_cond [K]

* Condenser temperature increase Dt_cond [K]

* Cooling demand Q_load [kJ/h]

* Isentropic efficiency N_is [-]

* Maximum chiller power Q_max [kJ/h]

* Control signal Ctrl [-]



In [2]:
data = pd.read_excel("senergy_nets_example_chiller.xlsx")
start = "2020-01-01 00:00:00"
end = "2020-01-01 01:59:59"
resol = 3600
dur = pd.date_range(start, end, freq="%ss" % resol)
data.index = dur

Similarly to pandapower and pandapipes, the input for a time-series calculation has to be converted to the DFData format, which we imported before.

In [3]:
data_DFD = DFData(data)

The needed set of *inputs* required to run the calculation of the chiller are the following:

1. **data_source** = the name of the file containing the time histories;
2. **profile_name_t_set_k** = Name of the column header of the Set Temperature column in the input data excel
3. **profile_name_t_in_ev_k** = Name of the column header of the Evaporator inlet temperature column in the input data excel
4. **profile_name_t_in_cond_k** = Name of the column header of the Condenser inlet temperature column in the input data excel
5. **profile_name_dt_cond_k** = Name of the column header of the Condenser temperature increase column in the input data excel
6. **profile_name_q_load_kj_per_h** = Name of the column header of the Cooling demand column in the input data excel
7. **profile_name_n_is** = Name of the column header of the Isentropic efficiency column in the input data excel
8. **profile_name_q_max_kj_per_h** = Name of the column header of the Maximum chiller power column in the input data excel
9. **profile_name_ctrl** = Name of the column header of the Control signal column in the input data excel

In the following example we also set a user-defined start and end of the time series, as well as the data_source. All the other optional inputs can be input when calling the chiller function.

In [4]:
prosumer_ch = chiller(
        period_start=start,
        period_end=end,
        data_source=data_DFD,
        profile_name_t_set_k=["Set Point Temperature T_set [K]"],
        profile_name_t_in_ev_k=["Evaporator inlet temperature T_in_ev [K]"],
        profile_name_t_in_cond_k=["Condenser inlet temperature T_in_cond [K]"],
        profile_name_dt_cond_k=["Condenser temperature increase Dt_cond [K]"],
        profile_name_q_load_kj_per_h=["Cooling demand Q_load [kJ/h]"],
        profile_name_n_is=["Isentropic efficiency N_is [-]"],
        profile_name_q_max_kj_per_h=["Maximum chiller power Q_max [kJ/h]"],
        profile_name_ctrl=["Control signal Ctrl [-]"],
  )


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 134.11it/s]


We can see what is the object *prosumer_ch*

In [5]:
prosumer_ch

Following constraints are included:
   - controller (1 entries)
   - location (1 entries)
   - period (1 entries)
   - sn_chiller (1 entries)
Following time_series are generated:
   - time_series (1 entries)

We can analyse the time series deeper, exporting them into a Pandas Dataframe.

In [6]:
res_df = prosumer_ch.time_series.loc[0].data_source.df
print(res_df)

                     q_evap_kj_per_h  unmet_load_kj_per_h  w_in_tot_kj_per_h  \
2020-01-01 00:00:00              0.0                  0.0           0.000000   
2020-01-01 01:00:00          18000.0                  0.0        4369.430404   

                         eer  plr  t_out_ev_k  t_out_cond_k  m_evap_kg_per_h  \
2020-01-01 00:00:00  0.00000  0.0         0.0           0.0         0.000000   
2020-01-01 01:00:00  4.11953  0.5       280.0         306.0       861.244019   

                     m_cond_kg_per_h  q_cond_kj_per_h  
2020-01-01 00:00:00         0.000000         0.000000  
2020-01-01 01:00:00      1801.146208     22586.373445  


The *outputs* that are produced are the following:

1. **q_evap_kj_per_h** = Chiller delivered power chiller
2. **unmet_load_kj_per_h** = Unmet demand
3. **w_in_tot_kj_per_h** = Compressor power consumption
4. **eer** = EER
5. **plr** = PLR
6. **t_out_ev_k** = Evaporator temperature
7. **t_out_cond_k** = Condenser temperature
8. **m_evap_kg_per_h** = Evaporator flow rate 
9. **m_cond_kg_per_h** = Condenser flow rate 
10. **q_cond_kj_per_h** = Evaporator power consumption 

The following default values are considered:

* **Period**
  
  1. *period_name* = 'default'
  2. *period_start* = '2020-01-01 00:00:00'
  3. *perod_end* = '2020-01-02 00:00:00'
  4. *period_resolution* = 3600

* **Location**
  
  1. *location_name* = 'cordoba'
  2. *location_lat*  =37.888
  3. *location_long* =4.77
  4. *elevation* = 0
  5. *timezone* = "utc"
  6. *state* = "HE"
  7. *country* = "Spain"
 
 
* **Characteristics of the Chiller**
  
  1. *cp_wat* = 4.18
  2. *t_sh* = 5.0
  3. *t_sc* = 2.0
  4. *pp_cond* = 5.0
  5. *pp_evap* = 5.0
  6. *plf_cc* = 0.9
  7. *w_evap_pump* = 200.0
  8. *w_cond_pump* = 200.0
  9. *eng_eff* = 1.0
  10. *n_ref* = "R410A"
   

As an example, we can assess another chiller, with the same time series and characteristics of the chiller, but considering water as refrigerant.

In [7]:
prosumer_ch_water = chiller(
        period_start=start,
        period_end=end,
        data_source=data_DFD,
        profile_name_t_set_k=["Set Point Temperature T_set [K]"],
        profile_name_t_in_ev_k=["Evaporator inlet temperature T_in_ev [K]"],
        profile_name_t_in_cond_k=["Condenser inlet temperature T_in_cond [K]"],
        profile_name_dt_cond_k=["Condenser temperature increase Dt_cond [K]"],
        profile_name_q_load_kj_per_h=["Cooling demand Q_load [kJ/h]"],
        profile_name_n_is=["Isentropic efficiency N_is [-]"],
        profile_name_q_max_kj_per_h=["Maximum chiller power Q_max [kJ/h]"],
        profile_name_ctrl=["Control signal Ctrl [-]"],
        n_ref = 'Water'
  )

res_water_df = prosumer_ch_water.time_series.loc[0].data_source.df
print(res_water_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 243.73it/s]

                     q_evap_kj_per_h  unmet_load_kj_per_h  w_in_tot_kj_per_h  \
2020-01-01 00:00:00              0.0                  0.0           0.000000   
2020-01-01 01:00:00          18000.0                  0.0        4274.616708   

                          eer  plr  t_out_ev_k  t_out_cond_k  m_evap_kg_per_h  \
2020-01-01 00:00:00  0.000000  0.0         0.0           0.0         0.000000   
2020-01-01 01:00:00  4.210904  0.5       280.0         306.0       861.244019   

                     m_cond_kg_per_h  q_cond_kj_per_h  
2020-01-01 00:00:00         0.000000         0.000000  
2020-01-01 01:00:00      1792.829217     22482.078379  


Let us consider now a new chiller, chiller 1

In [8]:
data_1 = pd.read_excel("senergy_nets_chiller1.xlsx")
start = "2020-01-01 00:00:00"
end = "2020-01-01 00:59:59"
resol = 3600
dur = pd.date_range(start, end, freq="%ss" % resol)
data_1.index = dur
data_DFD_1 = DFData(data_1)
prosumer_ch_1 = chiller(
        period_start=start,
        period_end=end,
        data_source=data_DFD_1,
        profile_name_t_set_k=["Set Point Temperature T_set [K]"],
        profile_name_t_in_ev_k=["Evaporator inlet temperature T_in_ev [K]"],
        profile_name_t_in_cond_k=["Condenser inlet temperature T_in_cond [K]"],
        profile_name_dt_cond_k=["Condenser temperature increase Dt_cond [K]"],
        profile_name_q_load_kj_per_h=["Cooling demand Q_load [kJ/h]"],
        profile_name_n_is=["Isentropic efficiency N_is [-]"],
        profile_name_q_max_kj_per_h=["Maximum chiller power Q_max [kJ/h]"],
        profile_name_ctrl=["Control signal Ctrl [-]"],
        cp_water = 4.18,
        t_sh = 5.0,
        t_sc = 2.0,
        pp_cond = 5.0,
        pp_evap = 5.0,
        plf_cc = 1.0,
        w_evap_pump = 900.0,
        w_cond_pump = 900.0,
        eng_eff = 0.99,
        n_ref = "R410A"
  )

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 213.26it/s]


In [9]:
res_chiller_1_df = prosumer_ch_1.time_series.loc[0].data_source.df
print(res_chiller_1_df)

            q_evap_kj_per_h  unmet_load_kj_per_h  w_in_tot_kj_per_h       eer  \
2020-01-01         337320.0                  0.0       78423.948671  4.301237   

            plr  t_out_ev_k  t_out_cond_k  m_evap_kg_per_h  m_cond_kg_per_h  \
2020-01-01  1.0      280.15        308.15     16139.712919     19769.268382   

            q_cond_kj_per_h  
2020-01-01    413177.709184  


In [11]:
data_2 = pd.read_excel("senergy_nets_chiller2.xlsx")
start = "2020-01-01 00:00:00"
end = "2020-01-01 00:59:59"
resol = 3600
dur = pd.date_range(start, end, freq="%ss" % resol)
data_2.index = dur
data_DFD_2 = DFData(data_2)
prosumer_ch_2 = chiller(
        period_start=start,
        period_end=end,
        data_source=data_DFD_2,
        profile_name_t_set_k=["Set Point Temperature T_set [K]"],
        profile_name_t_in_ev_k=["Evaporator inlet temperature T_in_ev [K]"],
        profile_name_t_in_cond_k=["Condenser inlet temperature T_in_cond [K]"],
        profile_name_dt_cond_k=["Condenser temperature increase Dt_cond [K]"],
        profile_name_q_load_kj_per_h=["Cooling demand Q_load [kJ/h]"],
        profile_name_n_is=["Isentropic efficiency N_is [-]"],
        profile_name_q_max_kj_per_h=["Maximum chiller power Q_max [kJ/h]"],
        profile_name_ctrl=["Control signal Ctrl [-]"],
        cp_water = 4.18,
        t_sh = 5.0,
        t_sc = 2.0,
        pp_cond = 5.0,
        pp_evap = 5.0,
        plf_cc = 1.0,
        w_evap_pump = 1980.0,
        w_cond_pump = 1980.0,
        eng_eff = 0.99,
        n_ref = "R1234ZE"
  )

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 180.93it/s]


In [12]:
res_chiller_2_df = prosumer_ch_2.time_series.loc[0].data_source.df
print(res_chiller_2_df)

            q_evap_kj_per_h  unmet_load_kj_per_h  w_in_tot_kj_per_h       eer  \
2020-01-01        1594800.0                  0.0      321552.483792  4.959688   

            plr  t_out_ev_k  t_out_cond_k  m_evap_kg_per_h  m_cond_kg_per_h  \
2020-01-01  1.0      280.15        308.15     76306.220096     91350.074591   

            q_cond_kj_per_h  
2020-01-01     1.909217e+06  
